<a href="https://colab.research.google.com/github/shreyasseshadri/SC-Project/blob/master/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [24]:
# # Load the Drive helper and mount
from google.colab import drive

# # This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install unidecode

In [3]:
import pandas as pd
from unidecode import unidecode

base_path="drive/My Drive/SC-Project"
train_pos=pd.read_csv(base_path+'/train/pos.csv')
train_neg=pd.read_csv(base_path+'/train/neg.csv')
test_pos=pd.read_csv(base_path+'/test/pos.csv')
test_neg=pd.read_csv(base_path+'/test/neg.csv')
print('imported data.....')

def clear_text(text):
    text=unidecode(unicode(text, encoding = "utf-8"))
    return text.replace('"', '').replace("'", "").replace("<br />","").replace(",","").replace("(","").replace(")","").replace("-","").replace("/","").replace("!","").encode("ascii", "ignore")

train_pos_sent=train_pos['text'].apply(clear_text).tolist()
train_neg_sent=train_neg['text'].apply(clear_text).tolist()
test_pos_sent=test_pos['text'].apply(clear_text).tolist()
test_neg_sent=test_neg['text'].apply(clear_text).tolist()


corpus_train=train_pos_sent+train_neg_sent
corpus_test=test_pos_sent+test_neg_sent


imported data.....


In [4]:
!pip install nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
from __future__ import print_function
def create_corpus(corpus):
    corpus_out=[]
    sw=stopwords.words('english')
    count=1
    new_reviews=[]
    for review in corpus:
        review_sents=review.split(".")
        new_review_sent=[]
        for sent in review_sents:
            temp=sent.split(" ")
            for word in temp:
                if word=='' or word==' ' or word in sw:
                    temp.remove(word)
            new_review_sent+=temp
            corpus_out.append(temp)
        new_reviews.append(new_review_sent)
        s=str(count)+'/'+str(len(corpus))+' done!'
        if count%1==0:
          print(s,end='\r')
        count+=1
    return corpus_out,new_reviews

In [5]:
train_wv,train_sent=create_corpus(corpus_train)

In [6]:
test_wv,test_sent=create_corpus(corpus_test)

In [7]:
!pip install gensim

In [53]:
# from gensim.models import Word2Vec
# model = Word2Vec(train_wv+test_wv, min_count=1,seed=123)
# print(model)

Word2Vec(vocab=213207, size=100, alpha=0.025)


In [61]:
# words = list(model.wv.vocab)
# print(words[:19]+['dscd'])

['', 'Feijo', 'selfexplanatorywhich', 'Craziness', 'RollsRoyce', 'EXPLAIN', 'sunflowers?', 'fleeces', 'considered?', 'woods', 'spiders', 'nohopers', 'considered:', 'ultimatley', 'woody', 'trawling', 'goodbetter', 'comically', 'spidery', 'dscd']


In [57]:
# model.save('drive/My Drive/SC-Project/model.bin')
# print("model saved")

model saved


In [0]:
from gensim.models import Word2Vec
model = Word2Vec.load('drive/My Drive/SC-Project/model.bin')

In [0]:
import numpy as np

In [0]:
def make_data(corpus,maxlen):
    data=[]
    for review in corpus:
        temp=model[review]
        if len(review)<maxlen:
            temp=np.concatenate((temp,np.zeros((maxlen-len(review),100))),0)
        else:
            temp=temp[:maxlen]
        data.append(temp)
    return np.array(data)

In [12]:
train_data=make_data(train_sent,150)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [13]:
test_data=make_data(test_sent,150)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [0]:
train_data=np.expand_dims(train_data,3)
test_data=np.expand_dims(test_data,3)

In [15]:
train_y=[1 for _ in range(len(train_pos_sent))]+[0 for _ in range(len(train_neg_sent))]
test_y=[1 for _ in range(len(test_pos_sent))]+[0 for _ in range(len(test_neg_sent))]
from keras.utils import np_utils
y_train=np_utils.to_categorical(train_y)
y_test=np_utils.to_categorical(test_y)

Using TensorFlow backend.


In [0]:
np.random.seed(123)
train_indices = np.arange(train_data.shape[0])
test_indices= np.arange(test_data.shape[0])
np.random.shuffle(train_indices)
np.random.shuffle(test_indices)
train_data = train_data[train_indices]
y_train = y_train[train_indices]
test_data = test_data[test_indices]
y_test = y_test[test_indices]

In [0]:
np.save('drive/My Drive/SC-Project/train_data.npy',train_data)
np.save('drive/My Drive/SC-Project/test_data.npy',test_data)
np.save('drive/My Drive/SC-Project/y_train.npy',y_train)
np.save('drive/My Drive/SC-Project/y_test.npy',y_test)